In [1]:
import requests
from bs4 import BeautifulSoup
import csv

url = "https://www.comebuy2002.com.tw/store/"
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
articles = soup.find_all("li", class_="g-mv-12 clearfix")

stores = []
for a in articles:
    store_name_tag = a.find("div", class_="itemBox storeTitle")
    store_name = store_name_tag.text.strip() if store_name_tag else "沒有店名"
    
    store_address_tag = a.find("div", class_="itemBox address")
    store_address = store_address_tag.text.strip() if store_address_tag else "沒有地址"
   
    store_phone_tag = a.find("div", class_="itemBox tel")
    store_phone = store_phone_tag.find("a").text.strip() if store_phone_tag and store_phone_tag.find("a") else "沒有電話"
    
    stores.append([store_name, store_address, store_phone])

# 將數據寫入 CSV 文件
csv_file = 'stores.csv'
with open(csv_file, mode='w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(["store_name", "store_address", "store_phone"])  # 寫入表頭
    writer.writerows(stores)

print(f"數據已成功寫入 {csv_file} 文件。")


數據已成功寫入 stores.csv 文件。


In [7]:
from flask import Flask, render_template_string, send_file
from flask_sqlalchemy import SQLAlchemy
import matplotlib.pyplot as plt
import pandas as pd
import io

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'mssql+pyodbc:///?' \
'driver=ODBC+Driver+17+for+SQL+Server&' \
'trusted_connection=yes&' \
'server=LAPTOP-QV5BBV9V&' \
'database=testdb1'
db = SQLAlchemy(app)



In [8]:
@app.route('/')
def index():
    return "資料庫連線成功！"

@app.route('/setup')
def setup():
    sql = '''
    IF OBJECT_ID(N'comebuy', N'U') IS NOT NULL
    DROP TABLE comebuy;
    CREATE TABLE comebuy (
    sid int NOT NULL IDENTITY(1,1) PRIMARY KEY,
    store_name nvarchar(50) NOT NULL,
    store_address nvarchar(200),
    store_phone nvarchar(50)
    )
    '''
    db.engine.execute(sql)
    return "資料表建立成功！"

@app.route('/insert')
def insert():
    with open('stores.csv', encoding='utf-8-sig') as file:
        reader = csv.reader(file)
        next(reader)  # 跳過表頭
        for row in reader:
            sql = f"""
            INSERT INTO comebuy (store_name, store_address, store_phone) VALUES('{row[0]}', '{row[1]}', '{row[2]}');
            """
            db.engine.execute(sql)
    return "資料新增成功！"

@app.route('/query')
def query():
    sql = "SELECT * FROM comebuy ORDER BY sid"
    stores = db.engine.execute(sql)
    msg = ""
    for store in stores:
        msg += f"{store['store_name']}, {store['store_address']}, {store['store_phone']}<br>"
    return msg

@app.route('/updateusr/<int:uid>')
def updateusr(uid):
    sql = "UPDATE comebuy SET store_name = '更新的店名' WHERE sid = " + str(uid)
    db.engine.execute(sql)
    return "資料修改成功！"

@app.route('/deleusr/<int:uid>')
def deleusr(uid):
    sql = "DELETE FROM comebuy WHERE sid = " + str(uid)
    db.engine.execute(sql)
    return "資料刪除成功！"

@app.route('/charts')
def charts():
    conn = db.engine.raw_connection()
    df = pd.read_sql("SELECT store_name, COUNT(*) as count FROM comebuy GROUP BY store_name", conn)
    conn.close()
    
    # 長條圖
    fig, ax = plt.subplots()
    df.plot(kind='bar', x='store_name', y='count', ax=ax)
    ax.set_xlabel("Store Name")
    ax.set_ylabel("Count")
    ax.set_title("Store Count by Name")
    buf_bar = io.BytesIO()
    plt.savefig(buf_bar, format='png')
    buf_bar.seek(0)
    
    # 圓餅圖
    fig, ax = plt.subplots()
    df.set_index('store_name').plot(kind='pie', y='count', ax=ax, autopct='%1.1f%%')
    ax.set_ylabel("")
    ax.set_title("Store Distribution")
    buf_pie = io.BytesIO()
    plt.savefig(buf_pie, format='png')
    buf_pie.seek(0)
    
    html = '''
    <!doctype html>
    <html lang="zh-Hant">
      <head>
        <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
        <title>Comebuy Charts</title>
      </head>
      <body>
        <div class="container">
          <h1 class="mt-5">Comebuy Charts</h1>
          <h2>長條圖</h2>
          <img src="data:image/png;base64,{}" alt="Bar Chart">
          <h2>圓餅圖</h2>
          <img src="data:image/png;base64,{}" alt="Pie Chart">
        </div>
      </body>
    </html>
    '''
    
    import base64
    bar_chart = base64.b64encode(buf_bar.getvalue()).decode('utf-8')
    pie_chart = base64.b64encode(buf_pie.getvalue()).decode('utf-8')
    
    return render_template_string(html.format(bar_chart, pie_chart))

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False)



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.104:5000
Press CTRL+C to quit


In [ ]:
import threading

def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False)

thread = threading.Thread(target=run_flask)
thread.start()



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.104:5000
Press CTRL+C to quit
